In [1]:
import nltk
import os 
from nltk.stem.lancaster import LancasterStemmer
import PIL
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

with open("intents.json") as file:
    data=json.load(file)

words=[]
labels=[]
doc_y=[]
doc_x=[]


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hrith\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        pattern=pattern.lower()

        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        doc_x.append(wrds)
        doc_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

stemmer=LancasterStemmer()
words=[stemmer.stem(w.lower()) for w in words if w not in "?"]
words=sorted(list(set(words)))
labels=sorted(labels)


In [4]:
out_empty = [0 for _ in range(len(labels))]
training=[]
output=[]
for x, doc in enumerate(doc_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(doc_y[x])] = 1
    training.append(bag)
    output.append(output_row)    
    
    #Converting our training data to numpy arrays
training = np.array(training)
output = np.array(output)

#Saving our data
with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

In [8]:
tf.compat.v1.reset_default_graph()
    
net= tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)   
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")    
net = tflearn.regression(net)
model= tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 7999  | total loss: 0.12420 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.12420 - acc: 0.9889 -- iter: 56/59
Training Step: 8000  | total loss: 0.11650 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.11650 - acc: 0.9900 -- iter: 59/59
--
INFO:tensorflow:c:\Users\hrith\OneDrive\Desktop\Project Manas\GDSC_Workshop\Chatbot_WS\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
